In [1]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, Var, PositiveReals, NonNegativeReals
from math import log
import pandas as pd


In [2]:
class ChemicalModel:
    def __init__(self):
        self.model = ConcreteModel()
        self.components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']
        self.parameters = Parameters()
        
        # Set params as an attribute of model
        self.model.params = self.parameters.params
        
        self.variables = Variables(self.model, self.components, self.model.params)
        self.constraints = Constraints(self.model, self.model.params)
        
    def solve(self):
        solver = SolverFactory('ipopt')
        solver.options['constr_viol_tol'] = 1e-8
        solver.options['acceptable_constr_viol_tol'] = 1e-8

        solver.solve(self.model, tee=True)

    def display_results(self):
        # Helper function to fetch the value
        def fetch_value(var):
            val = var.value
            if val is None or val < 0:
                return 0.0
            return val

        # Helper function to display and write a set of results
        def display_and_write(file, header, results):
            file.write(header + '\n')
            print(header)
            for result in results:
                file.write(result + '\n')
                print(result)

        # Specify the filename where you want to store the results
        filename = "model_results.txt"
        model = self.model 

        with open(filename, 'w') as file:
            # Write the header to the file
            file.write("Results:\n")
            print("Results:")

            # Stream S Results
            s_results = [
                f"S1: {model.params['S1']}",
                f"S2: {fetch_value(model.S2)}",
                f"S3: {fetch_value(model.S3)}",
                f"S4: {fetch_value(model.S4)}",
                f"S5: {fetch_value(model.S5)}",
                f"S6: {fetch_value(model.S6)}",
                f"S7: {fetch_value(model.S7)}"
            ]
            display_and_write(file, "\nStream S Results:", s_results)

            # d Composition Results
            components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']
            d_results = [
                f"d1[{component}]: {fetch_value(model.d1[component])}" for component in components
            ] + [
                f"d2[{component}]: {fetch_value(model.d2[component])}" for component in components
            ] + [
                f"d3[{component}]: {fetch_value(model.d3[component])}" for component in components
            ]
            display_and_write(file, "\nd Composition Results:", d_results)

            # b Composition Results
            b_results = [
                f"b1[{component}]: {fetch_value(model.b1[component])}" for component in components
            ] + [
                f"b2[{component}]: {fetch_value(model.b2[component])}" for component in components
            ] + [
                f"b3[{component}]: {fetch_value(model.b3[component])}" for component in components
            ]
            display_and_write(file, "\nb Composition Results:", b_results)

            # Generate the stream table
            stream_table_df = self.generate_stream_table()
            stream_table_str = stream_table_df.to_string()

            # Write the stream table to the file
            file.write("\nStream Table:\n")
            file.write(stream_table_str)
            file.write("\n")

        # Print to console that results are written to the file
        print(f"\nResults have been written to {filename}")


    def generate_stream_table(self):
        """Generate a table with molar flow rates of each component in each stream."""

        def fetch_value(var):
            val = var.value
            if val is None or val < 0:
                return 0.0
            return round(val, 4)  # Rounding to 4 decimal digits

        components = self.components
        data = []  # This will store rows of data which will be used to create DataFrame

        # For S1 (since it's defined in parameters)
        s1_flow = self.model.params['S1']
        s1_compositions = [self.model.params[f'z1_{component}'] for component in components]
        s1_molar_flow_rates = [s1_flow * comp for comp in s1_compositions]
        data.append(s1_molar_flow_rates)

        # For other S streams
        for i in range(2, 8):  # Start from S2 since we've already handled S1
            stream_name = "S" + str(i)
            s_flow = fetch_value(getattr(self.model, stream_name))

            if i % 2 == 0:  # Even numbered S streams
                stream_name = "d" + str(i//2)
                compositions = [fetch_value(getattr(self.model, stream_name)[component]) for component in components]
            else:  # Odd numbered S streams
                stream_name = "b" + str(i//2)
                compositions = [fetch_value(getattr(self.model, stream_name)[component]) for component in components]

            molar_flow_rates = [s_flow * comp for comp in compositions]
            data.append(molar_flow_rates)

        # Creating DataFrame
        stream_names = [f'S{i}' for i in range(1, 8)]
        df = pd.DataFrame(data, columns=components, index=stream_names)
        print(df)

        return df


In [3]:
class Constraints:
    components = ["Benzene", "Toluene", "OrthoXylene", "MetaXylene", "Ethylbenzene", "ParaXylene", "TwoMethylbutane"]

    def __init__(self, model, parameters=None):
        self.model = model
        if parameters:
            self.define_constraints(model, parameters)
        self.generate_all_constraints()

    def generate_all_constraints(self):
        for component in self.components:
            self.material_balance_rule1(component)
            self.material_balance_rule2(component)
            self.material_balance_rule3(component)

    def material_balance_rule1(self, component):
        def rule(model):
            return getattr(model, f'{component}_S1') == getattr(model, f'{component}_S2') + getattr(model, f'{component}_S3')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule1_constraint', self.model.Constraint(rule=rule))

    def material_balance_rule2(self, component):
        def rule(model):
            return getattr(model, f'{component}_S3') == getattr(model, f'{component}_S4') + getattr(model, f'{component}_S5')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule2_constraint', self.model.Constraint(rule=rule))

    def material_balance_rule3(self, component):
        def rule(model):
            return getattr(model, f'{component}_S5') == getattr(model, f'{component}_S6') + getattr(model, f'{component}_S7')

        # Adding the constraint to the model
        setattr(self.model, f'{component}_rule3_constraint', self.model.Constraint(rule=rule))


    def define_constraints(self, model, parameters):
        # Map each constraint to its corresponding function
        constraints_mapping = {
            'Eq1': self.Eq1,
            'Eq2': self.Eq2,
            'Eq3': self.Eq3,
            'Eq4': self.Eq4,
            'Eq5': self.Eq5,
            'Eq6': self.Eq6,
            'Eq7': self.Eq7,
            'Eq8': self.Eq8,
            'Eq9': self.Eq9,
            'Eq10': self.Eq10,  
            'Eq11': self.Eq11
        }

        components = ['Benzene', 'Toluene', 'OrthoXylene', 'MetaXylene', 'Ethylbenzene', 'ParaXylene', 'TwoMethylbutane']

        for component in components:
            # Dynamically create constraint name based on the component
            for i in range(1, 7):
                rule_name = f'{component}_comp_rule{i}'
                if rule_name in dir(self):
                    constraints_mapping[rule_name] = getattr(self, rule_name)

        # Dynamically add each constraint to the model
        for constraint_name, rule_function in constraints_mapping.items():
            setattr(model, constraint_name, Constraint(rule=rule_function))

          
    # Equations   
    def Eq1(self, model):
        return model.params['FR_S2_LK'] * model.params['S1'] * model.params['z1_Benzene'] - model.d1['Benzene'] * model.S2 == 0
    
    def Eq2(self, model):
        return (1 - model.params['FR_S2_LK']) * model.params['S1'] * model.params['z1_Benzene'] - model.b1['Benzene'] * model.S3 == 0
    
    def Eq3(self, model):
        return model.params['FR_S3_HK'] * model.params['S1'] * model.params['z1_Toluene'] == model.b1['Toluene'] * model.S3
    
    def Eq4(self, model):
        return (1 - model.params['FR_S3_HK']) * model.params['S1'] * model.params['z1_Toluene'] == model.d1['Toluene'] * model.S2
    
    def Eq5(self, model):
        return model.params['FR_S4_LK'] * model.S3 * model.b1['Toluene'] - model.d2['Toluene'] * model.S4 == 0
    
    def Eq6(self, model):
        return (1 - model.params['FR_S4_LK']) * model.S3 * model.b1['Toluene'] - model.b2['Toluene'] * model.S5 == 0
    
    def Eq7(self, model):
        return model.params['FR_S5_HK'] * model.S3 * model.b1['Ethylbenzene'] - model.b2['Ethylbenzene'] * model.S5 == 0
    
    def Eq8(self, model):
        return (1 - model.params['FR_S5_HK']) * model.S3 * model.b1['Ethylbenzene'] - model.d2['Ethylbenzene'] * model.S5 == 0
    
    def Eq9(self, model):
        return model.params['FR_S6_LK'] * model.S5 * model.b2['Ethylbenzene'] - model.d3['Ethylbenzene'] * model.S6 == 0
    
    def Eq10(self, model):
        return (1 - model.params['FR_S6_LK']) * model.S5 * model.b2['Ethylbenzene'] - model.b3['Ethylbenzene'] * model.S7 == 0
    
    def Eq11(self, model):
        return model.params['FR_S7_HK'] * model.S5 * model.b2['ParaXylene'] - model.b3['ParaXylene'] * model.S7 == 0
    
    def Eq12(self, model):
        return (1 - model.params['FR_S7_HK']) * model.S5 * model.b2['ParaXylene'] - model.d3['ParaXylene'] * model.S6 == 0


    
    # Benzene (Individual Component material balance)
    def Benzene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Benzene'] ==  model.d1['Benzene'] * model.S2 + model.b1['Benzene'] * model.S3

    def Benzene_comp_rule2(self, model):
        return model.S3 * model.b1['Benzene'] ==  model.S4 * model.d2['Benzene']
    
    def Benzene_comp_rule3(self, model):
        return model.b2['Benzene'] == 0
    
    def Benzene_comp_rule4(self, model):
        return model.d3['Benzene'] == 0
    
    def Benzene_comp_rule5(self, model):
        return model.b3['Benzene'] == 0
    
    # Toluene (Individual Component material balance)
    def Toluene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Toluene'] ==  model.d1['Toluene'] * model.S2 + model.b1['Toluene'] * model.S3
    
    def Toluene_comp_rule2(self, model):
        return model.S3 * model.b1['Toluene']  ==  model.S4 * model.d2['Toluene']  + model.S5 * model.b2['Toluene']
    
    def Toluene_comp_rule3(self, model):
        return model.b2['Toluene'] * model.S5 ==  model.S6 * model.d3['Toluene']

    def Toluene_zero_rule4(self, model):
        return model.b3['Toluene'] == 0

    
    
    # Ortho-Xylene (Individual Component material balance)
    def OrthoXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_OrthoXylene'] == model.b1['OrthoXylene'] * model.S3
    
    def OrthoXylene_comp_rule2(self, model):
        return model.d1['OrthoXylene'] == 0
    
    def OrthoXylene_comp_rule3(self, model):
        return model.b1['OrthoXylene'] * model.S3 == model.b2['OrthoXylene'] * model.S5
    
    def OrthoXylene_comp_rule4(self, model):
        return model.d2['OrthoXylene'] == 0

    def OrthoXylene_comp_rule5(self, model):
        return model.b2['OrthoXylene'] * model.S5 == model.b3['OrthoXylene'] * model.S7

    def OrthoXylene_comp_rule6(self, model):
        return model.d3['OrthoXylene'] == 0

    
    # Ethylbenzene (Individual Component material balance)
    def Ethylbenzene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_Ethylbenzene'] == model.b1['Ethylbenzene'] * model.S3  
    
    def Ethylbenzene_comp_rule2(self, model):
        return model.b1['Ethylbenzene'] * model.S3  == model.d2['Ethylbenzene'] * model.S4 + model.b2['Ethylbenzene'] * model.S5

    def Ethylbenzene_comp_rule3(self, model):
        return model.b2['Ethylbenzene'] * model.S5  == model.d3['Ethylbenzene'] * model.S6 + model.b3['Ethylbenzene'] * model.S7

    def Ethylbenzene_comp_rule4(self, model):
        return model.d1['Ethylbenzene'] == 0

    
    # MetaXylene (Individual Component material balance)
    def MetaXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_MetaXylene'] == model.b1['MetaXylene'] * model.S3
    
    def MetaXylene_comp_rule2(self, model):
        return model.d1['MetaXylene'] == 0
    
    def MetaXylene_comp_rule3(self, model):
        return model.b1['MetaXylene'] * model.S3 == model.b2['MetaXylene'] * model.S5
    
    def MetaXylene_comp_rule4(self, model):
        return model.d2['MetaXylene'] == 0

    def MetaXylene_comp_rule5(self, model):
        return model.b2['MetaXylene'] * model.S5 == model.b3['MetaXylene'] * model.S7

    def MetaXylene_comp_rule6(self, model):
        return model.d3['MetaXylene'] == 0


    
    # ParaXylene (Individual Component material balance)
    def ParaXylene_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_ParaXylene'] == model.b1['ParaXylene'] * model.S3 
    
    def ParaXylene_comp_rule2(self, model):
        return model.d1['ParaXylene'] == 0
    
    def ParaXylene_comp_rule3(self, model):
        return model.b1['ParaXylene'] * model.S3  ==  model.b2['ParaXylene'] * model.S5
    
    def ParaXylene_comp_rule4(self, model):
        return model.d2['ParaXylene'] == 0

    def ParaXylene_comp_rule5(self, model):
        return model.b2['ParaXylene'] * model.S5  == model.d3['ParaXylene'] * model.S6 + model.b3['ParaXylene'] * model.S7


    # TwoMethylbutane (Individual Component material balance)
    def TwoMethylbutane_comp_rule1(self, model):
        return model.params['S1'] * model.params['z1_TwoMethylbutane'] == model.d1['TwoMethylbutane'] * model.S2
    
    def TwoMethylbutane_comp_rule1(self, model):
        return model.b1['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule2(self, model):
        return model.d2['TwoMethylbutane'] == 0
        
    def TwoMethylbutane_comp_rule3(self, model):
        return model.b2['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule4(self, model):
        return model.d3['TwoMethylbutane'] == 0

    def TwoMethylbutane_comp_rule5(self, model):
        return model.b3['TwoMethylbutane'] == 0
    

In [4]:
class Parameters:
    def __init__(self):
        self.params = {
            # Molar flow rate of Stream 1 (Feed inlet)
            'S1': 4542.305, # to the power of 10**6 [kmol / hr]
            
            # Molar composition of Stream 1 (Feed inlet)
            'z1_Benzene': 0.2540,
            'z1_Toluene': 0.5840,
            'z1_OrthoXylene': 0.1374,
            'z1_MetaXylene': 0.0244,
            'z1_Ethylbenzene': 0.0,
            'z1_ParaXylene': 0.0001,
            'z1_TwoMethylbutane': 0.0,
            
            # Fractional recoveries of HK/LK
            'FR_S2_LK': 0.98,
            'FR_S3_HK': 0.95,
            
            'FR_S4_LK': 0.98,
            'FR_S5_HK': 0.95,
            
            'FR_S6_LK': 0.98,
            'FR_S7_HK': 0.95,
            
        }

In [5]:
class Variables:
    def __init__(self, model, components, parameters):
        self.define_variables(model, components, parameters)

    def define_variables(self, model, components, parameters):

        initial_value = 0.00001

        # Stream variables
        for i in range(2, 8):  # Streams from S2 to S7
            setattr(model, f'S{i}', Var(within=PositiveReals, initialize=parameters['S1']))

        # molar composition of Feed
        model.z = Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value)

        # molar compositions for Distillate and Bottom streams
        for i in range(1, 4):  # 3 sets of Distillate and Bottom streams
            setattr(model, f'd{i}', Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value))
            setattr(model, f'b{i}', Var(components, within=NonNegativeReals, bounds=(0, 1), initialize=initial_value))



In [6]:
chemical_model = ChemicalModel()

In [7]:
chemical_model.solve()

Ipopt 3.14.12: constr_viol_tol=1e-08
acceptable_constr_viol_tol=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:      121
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:       46
                     variables with only lower bounds:        6
                variables with lower and upper bounds:       40
                     variables with only upper bounds:        0
Total number of equal

 183r 0.0000000e+00 1.40e-02 1.00e+03  -3.5 0.00e+00  -9.5 0.00e+00 0.00e+00R  1
 184r 0.0000000e+00 3.32e-03 2.65e+04  -3.5 9.68e+00    -  8.10e-01 9.90e-04f  1
 185r 0.0000000e+00 1.92e-02 2.01e+04  -3.5 7.81e+00    -  2.98e-01 6.24e-01f  1
 186r 0.0000000e+00 1.84e-02 3.22e+03  -3.5 9.82e+00    -  1.00e+00 8.71e-01f  1
 187r 0.0000000e+00 1.53e-01 3.07e+01  -3.5 1.31e+01    -  1.00e+00 1.00e+00h  1
 188r 0.0000000e+00 2.39e-04 4.92e-02  -3.5 6.22e-01    -  1.00e+00 1.00e+00h  1
 189  0.0000000e+00 1.78e-05 2.11e+00  -3.8 8.07e-05    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190  0.0000000e+00 2.74e-05 1.13e+00  -3.8 3.26e-01    -  1.00e+00 1.00e+00H  1
 191  0.0000000e+00 3.15e-05 1.75e+03  -3.8 2.63e+02    -  1.00e+00 1.95e-03h 10
 192  0.0000000e+00 3.92e-05 1.67e+03  -3.8 5.94e+02 -10.0 1.00e+00 4.88e-04h 12
 193  0.0000000e+00 4.82e-05 1.67e+03  -3.8 1.45e+02 -10.5 1.00e+00 1.95e-03h 10
 194  0.0000000e+00 6.00e-05

 389  0.0000000e+00 5.84e-02 2.14e+03  -3.8 7.61e+02    -  1.00e+00 9.77e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390  0.0000000e+00 5.83e-02 2.15e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
 391  0.0000000e+00 5.83e-02 2.15e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
 392  0.0000000e+00 5.82e-02 2.16e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
 393  0.0000000e+00 4.21e+01 3.34e+02  -3.8 7.63e+02    -  1.00e+00 1.00e+00w  1
 394  0.0000000e+00 8.95e+00 1.90e+02  -3.8 1.61e+02    -  1.00e+00 8.94e-01w  1
 395  0.0000000e+00 1.06e+01 4.61e+03  -3.8 4.67e+02    -  1.00e+00 6.20e-01w  1
 396  0.0000000e+00 5.82e-02 2.16e+03  -3.8 4.01e+01  -6.2 1.00e+00 9.77e-04h 10
 397  0.0000000e+00 5.81e-02 2.16e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
 398  0.0000000e+00 5.81e-02 2.16e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
 399  0.0000000e+00 5.80e-02 2.17e+03  -3.8 7.62e+02    -  1.00e+00 9.77e-04h 11
iter    objective    inf_pr 

 552  0.0000000e+00 5.22e-02 2.29e+03  -3.8 6.25e+00  -5.7 1.00e+00 9.77e-04h 10
 553  0.0000000e+00 5.22e-02 2.29e+03  -3.8 6.93e+02    -  1.00e+00 9.77e-04h 11
 554  0.0000000e+00 5.21e-02 2.29e+03  -3.8 6.93e+02    -  1.00e+00 9.77e-04h 11
 555  0.0000000e+00 5.21e-02 2.30e+03  -3.8 6.92e+02    -  1.00e+00 9.77e-04h 11
 556  0.0000000e+00 5.20e-02 2.30e+03  -3.8 6.91e+02    -  1.00e+00 9.77e-04h 11
 557  0.0000000e+00 5.20e-02 2.30e+03  -3.8 6.91e+02    -  1.00e+00 9.77e-04h 11
 558  0.0000000e+00 5.19e-02 2.30e+03  -3.8 6.90e+02    -  1.00e+00 9.77e-04h 11
 559  0.0000000e+00 5.19e-02 2.30e+03  -3.8 6.90e+02    -  1.00e+00 9.77e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560  0.0000000e+00 5.18e-02 2.30e+03  -3.8 6.89e+02    -  1.00e+00 9.77e-04h 11
 561  0.0000000e+00 5.18e-02 2.30e+03  -3.8 6.89e+02    -  1.00e+00 9.77e-04h 11
 562  0.0000000e+00 3.77e+01 3.19e+02  -3.8 6.88e+02    -  1.00e+00 1.00e+00w  1
 563  0.0000000e+00 7.68e+00

 713  0.0000000e+00 4.66e-02 2.43e+03  -3.8 6.37e+02    -  1.00e+00 9.77e-04h 11
 714  0.0000000e+00 4.65e-02 2.43e+03  -3.8 6.36e+02    -  1.00e+00 9.77e-04h 11
 715  0.0000000e+00 4.65e-02 2.43e+03  -3.8 6.36e+02    -  1.00e+00 9.77e-04h 11
 716  0.0000000e+00 4.65e-02 2.43e+03  -3.8 6.35e+02    -  1.00e+00 9.77e-04h 11
 717  0.0000000e+00 4.64e-02 2.43e+03  -3.8 6.35e+02    -  1.00e+00 9.77e-04h 11
 718  0.0000000e+00 3.56e+01 3.06e+02  -3.8 6.35e+02    -  1.00e+00 1.00e+00w  1
 719  0.0000000e+00 6.93e+00 1.04e+02  -3.8 2.38e+02    -  1.00e+00 9.14e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 720  0.0000000e+00 5.41e-01 1.11e+03  -3.8 3.82e+00  -6.2 1.00e+00 9.22e-01w  1
 721  0.0000000e+00 4.64e-02 2.43e+03  -3.8 8.16e-01  -5.7 1.00e+00 9.77e-04h 10
 722  0.0000000e+00 4.63e-02 2.43e+03  -3.8 6.34e+02    -  1.00e+00 9.77e-04h 11
 723  0.0000000e+00 4.63e-02 2.43e+03  -3.8 6.34e+02    -  1.00e+00 9.77e-04h 11
 724  0.0000000e+00 4.62e-02

...

2233  0.0000000e+00 1.10e-01 9.16e-02  -5.7 4.48e+00  -9.9 1.00e+00 1.00e+00s 22
2234  0.0000000e+00 2.95e-06 3.74e-03  -5.7 2.22e-01  -8.5 1.00e+00 1.00e+00s 22
2235r 0.0000000e+00 2.95e-06 9.72e+02  -5.7 0.00e+00  -9.0 0.00e+00 0.00e+00R  1
2236r 0.0000000e+00 1.59e-08 1.79e+02  -5.7 6.00e-06  -4.0 1.00e+00 2.86e-02f  1
2237  0.0000000e+00 7.32e-05 3.56e-02  -5.7 1.21e-01  -9.5 1.00e+00 1.00e+00h  1
2238  0.0000000e+00 2.79e-01 5.02e+00  -5.7 7.50e+00  -9.1 1.00e+00 1.00e+00h  1
2239  0.0000000e+00 2.22e-01 9.22e+00  -5.7 7.31e+00  -9.5 1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2240  0.0000000e+00 1.16e-01 9.99e+00  -5.7 1.50e+00  -9.1 1.00e+00 5.00e-01h  2
2241  0.0000000e+00 1.01e-01 1.56e+01  -5.7 5.72e-01  -9.6 1.00e+00 1.25e-01h  4
2242  0.0000000e+00 8.85e-02 1.56e+01  -5.7 2.76e-01  -9.2 1.00e+00 1.25e-01h  4
2243  0.0000000e+00 6.21e-05 9.38e-03  -5.7 4.78e-02  -8.7 1.00e+00 1.00e+00h  1
2244  0.0000000e+00 6.21e-05

2403  0.0000000e+00 4.48e-05 1.09e+01  -5.7 1.72e+00  -9.1 1.00e+00 1.95e-03h 10
2404  0.0000000e+00 4.48e-05 1.24e+01  -5.7 2.87e+00  -9.6 1.00e+00 4.88e-04h 12
2405  0.0000000e+00 4.48e-05 1.11e+01  -5.7 1.84e+00  -9.1 1.00e+00 1.95e-03h 10
2406  0.0000000e+00 4.48e-05 9.82e+00  -5.7 9.46e-01  -8.7 1.00e+00 7.81e-03h  8
2407  0.0000000e+00 4.47e-05 1.12e+01  -5.7 1.98e+00  -9.2 1.00e+00 9.77e-04h 11
2408  0.0000000e+00 4.47e-05 9.96e+00  -5.7 1.04e+00  -8.8 1.00e+00 3.91e-03h  9
2409  0.0000000e+00 4.46e-05 1.13e+01  -5.7 2.10e+00  -9.2 1.00e+00 9.77e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2410  0.0000000e+00 6.41e-02 2.71e-01  -5.7 3.19e+00  -9.7 1.00e+00 1.00e+00w  1
2411  0.0000000e+00 1.51e-02 9.21e-02  -5.7 1.67e+00  -9.3 1.00e+00 1.00e+00w  1
2412  0.0000000e+00 3.10e-03 2.99e-02  -5.7 7.54e-01  -8.9 1.00e+00 1.00e+00w  1
2413  0.0000000e+00 4.46e-05 1.28e+01  -5.7 3.87e+00    -  1.00e+00 4.88e-04h 11
2414  0.0000000e+00 4.44e-05

2562  0.0000000e+00 3.90e-05 1.37e+01  -5.7 4.90e+00    -  1.00e+00 2.44e-04h 13
2563  0.0000000e+00 3.90e-05 9.57e+00  -5.7 1.77e+00  -9.0 1.00e+00 1.95e-03h 10
2564  0.0000000e+00 3.90e-05 8.24e+00  -5.7 8.66e-01  -8.6 1.00e+00 7.81e-03h  8
2565  0.0000000e+00 3.90e-05 9.68e+00  -5.7 1.92e+00  -9.1 1.00e+00 9.77e-04h 11
2566  0.0000000e+00 6.50e-02 2.16e-01  -5.7 3.23e+00  -9.6 1.00e+00 1.00e+00w  1
2567  0.0000000e+00 6.39e-02 1.21e-01  -5.7 3.38e+00 -10.0 1.00e+00 1.00e+00w  1
2568  0.0000000e+00 8.84e-03 2.99e-02  -5.7 1.34e+00  -9.6 1.00e+00 1.00e+00w  1
2569  0.0000000e+00 3.90e-05 1.15e+01  -5.7 1.44e+00 -10.1 1.00e+00 4.88e-04h 11
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2570  0.0000000e+00 3.90e-05 1.34e+01  -5.7 4.66e+00 -10.6 1.00e+00 2.44e-04h 13
2571  0.0000000e+00 3.90e-05 1.03e+01  -5.7 2.33e+00  -9.2 1.00e+00 9.77e-04h 11
2572  0.0000000e+00 3.90e-05 8.78e+00  -5.7 1.25e+00  -8.8 1.00e+00 3.91e-03h  9
2573  0.0000000e+00 3.88e-05

2709  0.0000000e+00 1.01e-06 3.27e+00  -5.7 3.28e-01  -9.3 1.00e+00 1.95e-03h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2710  0.0000000e+00 1.73e-03 3.22e-03  -5.7 5.83e-01  -9.8 1.00e+00 1.00e+00w  1
2711  0.0000000e+00 2.22e-04 7.22e-04  -5.7 2.12e-01  -9.4 1.00e+00 1.00e+00w  1
2712  0.0000000e+00 3.65e-05 1.97e-04  -5.7 8.55e-02  -8.9 1.00e+00 1.00e+00w  1
2713  0.0000000e+00 1.01e-06 3.27e+00  -5.7 1.99e-01  -9.4 1.00e+00 4.88e-04h 11
2714  0.0000000e+00 1.01e-06 3.26e+00  -5.7 1.90e-01  -9.0 1.00e+00 3.91e-03h  9
2715  0.0000000e+00 1.01e-06 3.17e+00  -5.7 8.13e-02  -8.6 1.00e+00 3.12e-02h  6
2716  0.0000000e+00 1.01e-06 3.24e+00  -5.7 2.09e-01  -9.0 1.00e+00 3.91e-03h  9
2717  0.0000000e+00 1.01e-06 3.25e+00  -5.7 4.37e-01  -9.5 1.00e+00 9.77e-04h 11
2718  0.0000000e+00 9.88e-07 2.85e+00  -5.7 3.62e-02  -8.2 1.00e+00 1.25e-01h  4
2719  0.0000000e+00 9.86e-07 3.14e+00  -5.7 1.04e-01  -8.7 1.00e+00 1.56e-02h  7
iter    objective    inf_pr 

2859  0.0000000e+00 8.58e-07 2.76e+00  -5.7 2.44e-01  -9.1 1.00e+00 1.95e-03h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2860  0.0000000e+00 8.59e-07 2.72e+00  -5.7 1.07e-01  -8.6 1.00e+00 1.56e-02h  7
2861  0.0000000e+00 8.41e-07 2.58e+00  -5.7 4.26e-02  -8.2 1.00e+00 6.25e-02h  5
2862  0.0000000e+00 8.39e-07 2.71e+00  -5.7 1.20e-01  -8.7 1.00e+00 7.81e-03h  8
2863  0.0000000e+00 8.39e-07 2.72e+00  -5.7 2.95e-01  -9.2 1.00e+00 1.95e-03h 10
2864  0.0000000e+00 8.39e-07 2.73e+00  -5.7 5.75e-01  -9.6 1.00e+00 4.88e-04h 12
2865  0.0000000e+00 8.39e-07 2.73e+00  -5.7 8.42e-01 -10.1 1.00e+00 2.44e-04h 13
2866  0.0000000e+00 1.10e-04 7.58e-04  -5.7 1.47e-01  -8.8 1.00e+00 1.00e+00w  1
2867  0.0000000e+00 6.38e-04 8.87e-04  -5.7 3.55e-01  -9.3 1.00e+00 1.00e+00w  1
2868  0.0000000e+00 7.99e-05 1.83e-04  -5.7 1.27e-01  -8.8 1.00e+00 1.00e+00w  1
2869  0.0000000e+00 8.39e-07 2.71e+00  -5.7 2.70e-01  -9.3 1.00e+00 7.81e-03h  7
iter    objective    inf_pr 

In [8]:
chemical_model.display_results()

Results:

Stream S Results:
S1: 4542.305
S2: 2305.966036294192
S3: 3187.2667978899635
S4: 4924.587340947766
S5: 847.4897108755546
S6: 100.8716025863169
S7: 956.4315722612812

d Composition Results:
d1[Benzene]: 0.49032403027802046
d1[Toluene]: 0.05751832590437966
d1[OrthoXylene]: 0.0
d1[MetaXylene]: 0.0
d1[Ethylbenzene]: 0.0
d1[ParaXylene]: 0.0
d1[TwoMethylbutane]: 1e-05
d2[Benzene]: 0.004685653396403996
d2[Toluene]: 0.5014977350863062
d2[OrthoXylene]: 0.0
d2[MetaXylene]: 0.0
d2[Ethylbenzene]: 0.0
d2[ParaXylene]: 0.0
d2[TwoMethylbutane]: 0.0
d3[Benzene]: 0.0
d3[Toluene]: 0.4996591209787817
d3[OrthoXylene]: 0.0
d3[MetaXylene]: 0.0
d3[Ethylbenzene]: 1.486939900079972e-15
d3[ParaXylene]: 0.00022515281226513298
d3[TwoMethylbutane]: 0.0

b Composition Results:
b1[Benzene]: 0.007239716930906473
b1[Toluene]: 0.7906682978871863
b1[OrthoXylene]: 0.195814391005226
b1[MetaXylene]: 0.03477344352640113
b1[Ethylbenzene]: 1.776822090716225e-17
b1[ParaXylene]: 0.00014251411281311939
b1[TwoMethylbutane

In [9]:
chemical_model.generate_stream_table()

        Benzene      Toluene  OrthoXylene  MetaXylene  Ethylbenzene  \
S1  1153.745470  2652.706120   624.112707  110.832242           0.0   
S2  1130.615130   132.593045     0.000000    0.000000           0.0   
S3    22.948321  2520.171859   624.066839  110.916885           0.0   
S4    23.145560  2469.680531     0.000000    0.000000           0.0   
S5     0.000000    50.425637   624.091415  110.851653           0.0   
S6     0.000000    50.405539     0.000000    0.000000           0.0   
S7     0.000000     0.000000   624.071619  110.850422           0.0   

    ParaXylene  TwoMethylbutane  
S1    0.454231              0.0  
S2    0.000000              0.0  
S3    0.318727              0.0  
S4    0.000000              0.0  
S5    0.423745              0.0  
S6    0.020174              0.0  
S7    0.478216              0.0  


,Benzene,Toluene,OrthoXylene,MetaXylene,Ethylbenzene,ParaXylene,TwoMethylbutane
S1,1153.745470,2652.706120,624.112707,110.832242,0.0,0.454231,0.0
S2,1130.615130,132.593045,0.000000,0.000000,0.0,0.000000,0.0
S3,22.948321,2520.171859,624.066839,110.916885,0.0,0.318727,0.0
S4,23.145560,2469.680531,0.000000,0.000000,0.0,0.000000,0.0
S5,0.000000,50.425637,624.091415,110.851653,0.0,0.423745,0.0
S6,0.000000,50.405539,0.000000,0.000000,0.0,0.020174,0.0
S7,0.000000,0.000000,624.071619,110.850422,0.0,0.478216,0.0


In 10^6 kmol / hr